In [1]:
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [2]:
client = boto3.client('s3')

In [3]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [4]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/data/processed/')

In [5]:
files = []
all_symbols = set()

In [6]:
# def calculate_ma(df, ema=True, sma=True, all_windows=[x for x in range(1, 240)]):
#     # Calculate EMAs and SMAs for each window
#     combined_df = pd.DataFrame()
#     for symbol in df.symbol.unique():
#         current_df = df[df.symbol==symbol]
#         print(symbol)
#         for window in tqdm(all_windows):
#             for price in ['open', 'high', 'low', 'close', 'volume']:
#                 current_df[f'{price}_ema_{window}m'] = current_df[price].ewm(span=window, adjust=False).mean()
#                 current_df[f'{price}_sma_{window}m'] = current_df[price].rolling(window=window).mean()
#         combined_df = pd.concat([combined_df, current_df])
#     df = combined_df
#     return df

In [ ]:
# for content in response.get('Contents', []):
#     current_file_dict = {}
#     current_file_dict['name'] = content['Key'].split('/')[-1].split('.')[0]
#     current_file_dict['loc'] = f"{s3_prefix}{bucket}/{content['Key']}"
#     df = pd.read_parquet(current_file_dict['loc'])
#     print(current_file_dict['name'])
#     # print(df.head(1))
#     df = calculate_ma(df)
#     df.to_parquet(f"{s3_prefix}{bucket}/{primary_folder}/data/data_prep/{current_file_dict['name']}.parquet")
#     print(f"saved to {s3_prefix}{bucket}/{primary_folder}/data/data_prep/{current_file_dict['name']}.parquet")
#     del df

In [16]:
all_windows = list(range(1, 240))

def calculate_ma(df, ema=True, sma=True, all_windows=all_windows):
    # Function to apply moving averages
    def apply_moving_averages(group):
        for window in tqdm(all_windows):
            for price in ['open', 'high', 'low', 'close', 'volume']:
                if ema:
                    group[f'{price}_ema_{window}m'] = group[price].ewm(span=window, adjust=False).mean()
                if sma:
                    group[f'{price}_sma_{window}m'] = group[price].rolling(window=window).mean()
        return group

    # Apply function by group
    return df.groupby('symbol').apply(apply_moving_averages)

In [19]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

current_run_paths = [x for x in paths if '240min' in x or '180min' in x or '120min' in x or '60min' in x]
# current_run_paths = [x for x in paths if '30min' in x or '25min' in x or '_20min' in x or '15min' in x or '10min' in x]
current_run_paths = [x for x in paths if '_1min' in x]
current_run_paths

['s3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_120min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_15min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_180min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_20min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_240min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_25min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_30min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_60min.parquet']

In [18]:
for content in current_run_paths:
    df = pd.read_parquet(content)
    print(content)
    df = calculate_ma(df)
    df.to_parquet(content.replace('processed', 'data_prep'))
    # df.to_parquet(f"{s3_prefix}{bucket}/{primary_folder}/data/data_prep/{current_file_dict['name']}.parquet")
    # print(f"saved to {s3_prefix}{bucket}/{primary_folder}/data/data_prep/{current_file_dict['name']}.parquet")
    # print(content.replace('processed', 'data_prep'))
    del df

s3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_100D.parquet


100%|██████████| 239/239 [00:02<00:00, 94.35it/s] 


s3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_10D.parquet


100%|██████████| 239/239 [00:02<00:00, 85.09it/s]


s3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_150D.parquet


...

100%|██████████| 239/239 [00:02<00:00, 117.99it/s]


s3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_50D.parquet


100%|██████████| 239/239 [00:02<00:00, 90.35it/s] 


s3://sisyphus-general-bucket/AthenaInsights/data/processed/stock_bars_5D.parquet


100%|██████████| 239/239 [00:02<00:00, 79.75it/s]


In [ ]:
# 3min, 5min, 

# all_windows = list(range(1, 240))

# def calculate_ma(df, ema=True, sma=True, all_windows=all_windows, price='open'):
#     # Function to apply moving averages
#     def apply_moving_averages(group):
#         for window in tqdm(all_windows):
#             if ema:
#                 group[f'{price}_ema_{window}m'] = group[price].ewm(span=window, adjust=False).mean()
#             if sma:
#                 group[f'{price}_sma_{window}m'] = group[price].rolling(window=window).mean()
#         return group

#     # Apply function by group
#     return df.groupby('symbol').apply(apply_moving_averages)

# for content in current_run_paths:
#     for price in ['open', 'high', 'low', 'close', 'volume']:
#         df = pd.read_parquet(content)
#         print(content)
#         # print(df.head(1))
#         df = calculate_ma(df[[price, 'symbol']], price='open')
#         df.to_parquet(content.replace('processed', 'data_prep').replace('.parquet', f'_{price}.parquet'))
#         print(f"saved to {content.replace('processed', 'data_prep').replace('.parquet', f'_{price}.parquet')}")
#         # df.to_parquet(f"{s3_prefix}{bucket}/{primary_folder}/data/data_prep/{current_file_dict['name']}.parquet")
#         # print(f"saved to {s3_prefix}{bucket}/{primary_folder}/data/data_prep/{current_file_dict['name']}.parquet")
#         # print(content.replace('processed', 'data_prep'))
#         del df

In [ ]:
# # 1min, 2min

# all_windows = list(range(1, 240))

# def calculate_ma_for_year(df, year, ema=True, sma=True, all_windows=all_windows):
#     # Filter data for the specified year including last three days of previous year
#     start_date = pd.Timestamp(year=year, month=1, day=1, tz='US/Eastern') - pd.Timedelta(days=3)
#     end_date = pd.Timestamp(year=year, month=12, day=31, tz='US/Eastern')
#     # print(df.head())
#     year_data = df[(df.index >= start_date) & (df.index <= end_date)]


#     def apply_moving_averages(group):
#         # print(group)
#         for window in tqdm(all_windows):
#             if ema:
#                 group[f'{price}_ema_{window}m'] = group[price].ewm(span=window, adjust=False).mean()
#             if sma:
#                 group[f'{price}_sma_{window}m'] = group[price].rolling(window=window).mean()
#         return group

#     # Apply moving averages
#     return year_data.groupby('symbol').apply(apply_moving_averages)#.reset_index()#.set_index('us_eastern_timestamp')

# for content in current_run_paths:
#     # Process each price
#     for price in ['open', 'high', 'low', 'close', 'volume']:
#         for year in range(2020, 2024):  # Update as needed
#             # df_price = .copy()  # focus on one price at a time
#             df = pd.read_parquet(content)[[price, 'symbol']].ffill()
#             df = calculate_ma_for_year(df, year)
#             df.to_parquet(content.replace('processed', 'data_prep').replace('.parquet', f'_{price}_{year}.parquet'))
#             print(content.replace('processed', 'data_prep').replace('.parquet', f'_{price}_{year}.parquet'))
#         del df

#         #     result_dfs.append(df)

#         # # Concatenate results for this price across all years
#         # full_df = pd.concat(result_dfs)
        
#         # print(f"saved to {content.replace('processed', 'data_prep').replace('.parquet', f'_{price}.parquet')}")
#             # del df